In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
%matplotlib notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [14]:
f=np.array([0.5,0,-0.5])
b=np.array([0.9,1.1,1])
r=np.array([-0.05,0.15,0])

B=np.vstack([np.ones(3),b]).T
B

S=np.diag(np.ones(3)*0.2)
Sinv=np.diag(np.ones(3)*5)
# uncomment below to see what happens if S!=(sigma**2)*I
# S[-1,-1]=0.25
# Sinv[-1,-1]=4

array([[1. , 0.9],
       [1. , 1.1],
       [1. , 1. ]])

In [15]:
IC=np.corrcoef(f,r)
IC

array([[ 1.        , -0.24019223],
       [-0.24019223,  1.        ]])

Lagrange multipliers

In [16]:
l=np.linalg.inv(B.T.dot(Sinv).dot(B)).dot(B.T.dot(Sinv).dot(f))
l

array([ 2.5, -2.5])

Risk-adjusted Factors

In [17]:
F=(f-l.dot(B.T))/np.diag(S)
F

array([ 1.25,  1.25, -2.5 ])

Risk adjusted factors can be also derived from regression

In [18]:
results = sm.WLS(f, B,np.diag(Sinv)).fit()
results.summary()
results.resid/np.diag(S) # F

/home/alex/anaconda3/lib/python3.7/site-packages/statsmodels/stats/stattools.py:72: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  "samples were given." % int(n), ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.250
Model:                            WLS   Adj. R-squared:                 -0.500
Method:                 Least Squares   F-statistic:                    0.3333
Date:                Sat, 04 May 2019   Prob (F-statistic):              0.667
Time:                        08:57:00   Log-Likelihood:                -1.1377
No. Observations:                   3   AIC:                             6.275
Df Residuals:                       1   BIC:                             4.473
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.5000      4.345      0.575      0.668     -52.703      57.703
x1            -2.5000      4.330     -0.577      0.667     -57.519      52.519
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   1.500
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.531
Skew:                          -0.707   Prob(JB):                        0.767
Kurtosis:                       1.500   Cond. No.                         24.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

array([ 1.25,  1.25, -2.5 ])

Returns of the risk factors

In [80]:
m=np.linalg.inv(B.T.dot(Sinv).dot(B)).dot(B.T.dot(Sinv).dot(r))
m

array([-0.96666667,  1.        ])

Risk-adjusted Returns

In [81]:
R=(r-m.dot(B.T))/np.diag(S)
R
R.mean()

array([ 0.08333333,  0.08333333, -0.16666667])

2.0354088784794536e-16

In [83]:
IC_risk_adjusted=np.corrcoef(R,F)
IC_risk_adjusted

array([[1., 1.],
       [1., 1.]])

Note that to get the return of risk factors, you can regress stock returns on the loadings matrix. The coefficients of the regression are the returns of risk factors with const=m0 that will make avg(R)=0

In [94]:
np.diag(Sinv)**2

array([25., 25., 16.])

In [103]:
results = sm.OLS(r, B).fit()
results.summary()

/home/alex/anaconda3/lib/python3.7/site-packages/statsmodels/stats/stattools.py:72: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  "samples were given." % int(n), ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.923
Model:                            OLS   Adj. R-squared:                  0.846
Method:                 Least Squares   F-statistic:                     12.00
Date:                Sat, 20 Apr 2019   Prob (F-statistic):              0.179
Time:                        11:28:15   Log-Likelihood:                 6.9865
No. Observations:                   3   AIC:                            -9.973
Df Residuals:                       1   BIC:                            -11.78
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.9667      0.290     -3.338      0.185      -4.647       2.714
x1             1.0000      0.289      3.464      0.179      -2.668       4.668
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   1.500
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.531
Skew:                          -0.707   Prob(JB):                        0.767
Kurtosis:                       1.500   Cond. No.                         24.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [98]:
np.isclose(results.params,m)

array([False,  True])

In [99]:
F.mean()
F.std()
R.mean()
R.std()
R/F

0.0

1.7677669529663689

2.0354088784794536e-16

0.11785113019775764

array([0.06666667, 0.06666667, 0.06666667])

Verify equation 19 from https://ssrn.com/abstract=2965224 	
Decoding Stock Market with Quant Alphas

Dollar-neutrality constraint

In [68]:
Q=np.ones(3)
R_alpha_s=sm.OLS(r, Q).fit()
R_alpha_s.summary()
R_alpha_s.params
r.mean()

/home/alex/anaconda3/lib/python3.7/site-packages/statsmodels/stats/stattools.py:72: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  "samples were given." % int(n), ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Fri, 19 Apr 2019   Prob (F-statistic):                nan
Time:                        19:15:54   Log-Likelihood:                 3.1391
No. Observations:                   3   AIC:                            -4.278
Df Residuals:                       2   BIC:                            -5.180
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0333      0.060      0.555      0.635      -0.225       0.292
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   2.885
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.421
Skew:                           0.528   Prob(JB):                        0.810
Kurtosis:                       1.500   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

array([0.03333333])

0.03333333333333333

In [69]:
R_prime_A_s=r-Q*R_alpha_s.params[0]
R_prime_A_s
R_prime_A_s.dot(Q)

array([-0.08333333,  0.11666667, -0.03333333])

-6.938893903907228e-18

Verify above for both dollar-neutrality and beta-neutrality constraint

In [100]:
Q=B
R_alpha_s=sm.OLS(r, Q).fit()
R_alpha_s.summary()
R_alpha_s.params
r.mean()
R_alpha_s.resid

/home/alex/anaconda3/lib/python3.7/site-packages/statsmodels/stats/stattools.py:72: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  "samples were given." % int(n), ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.923
Model:                            OLS   Adj. R-squared:                  0.846
Method:                 Least Squares   F-statistic:                     12.00
Date:                Sat, 20 Apr 2019   Prob (F-statistic):              0.179
Time:                        11:26:34   Log-Likelihood:                 6.9865
No. Observations:                   3   AIC:                            -9.973
Df Residuals:                       1   BIC:                            -11.78
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.9667      0.290     -3.338      0.185      -4.647       2.714
x1             1.0000      0.289      3.464      0.179      -2.668       4.668
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   1.500
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.531
Skew:                          -0.707   Prob(JB):                        0.767
Kurtosis:                       1.500   Cond. No.                         24.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

array([-0.96666667,  1.        ])

0.03333333333333333

array([ 0.01666667,  0.01666667, -0.03333333])

In [101]:
R_prime_A_s=r-Q.dot(R_alpha_s.params)
R_prime_A_s
R_prime_A_s.dot(Q)

array([ 0.01666667,  0.01666667, -0.03333333])

array([-5.41233725e-16, -5.34294831e-16])

In [102]:
print(f"Qian's risk-adjusted returns: {R}")
print(f"Kukushadze's residualized wrt constraints returns from eqs 18,19\n (residuals of regression of realized returns onto constraints loading)\n {R_prime_A_s}")
print(f"raw returns {r}")
# you can test this by changin S by uncommenting S modifications in the top cells
print(f"Risk/constraint adjusted returns are scaled by the S-inv when S=(sigma**2)*I, i.e. when idiosyncratic variance is the same for all stocks: {R/R_prime_A_s}")
Sinv
print('Means')
R.mean()
R_prime_A_s.mean()
r.mean()
print('Standard Deviations')
R.std()
R_prime_A_s.std()
r.std()

Qian's risk-adjusted returns: [ 0.08333333  0.08333333 -0.16666667]
Kukushadze's residualized wrt constraints returns from eqs 18,19
 (residuals of regression of realized returns onto constraints loading)
 [ 0.01666667  0.01666667 -0.03333333]
raw returns [-0.05  0.15  0.  ]
Risk/constraint adjusted returns are scaled by the S-inv when S=(sigma**2)*I, i.e. when idiosyncratic variance is the same for all stocks: [5. 5. 5.]


array([[5., 0., 0.],
       [0., 5., 0.],
       [0., 0., 4.]])

Means


2.0354088784794536e-16

-1.8041124150158794e-16

0.03333333333333333

Standard Deviations


0.11785113019775764

0.02357022603955153

0.08498365855987976